# Data Science Lab Project

##Cell 1:
Mounting Google Drive and Loading CSV Files
This code block does three main things:

Mounts your Google Drive: It connects your Google Drive to the current environment (likely Google Colab) so you can access files stored there. This is done using the drive.mount() function.

In [ ]:
# Cell 1: Mount Drive & Load All Survey CSVs
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import pandas as pd, glob, os, re

DATA_DIR = '/content/drive/MyDrive/DataScienceLabProject'
csv_paths = (
    sorted(glob.glob(os.path.join(DATA_DIR, '[2][0][1-2][7-9].csv'))) +
    sorted(glob.glob(os.path.join(DATA_DIR, '202[0-4].csv')))
)

dfs = {}
for path in csv_paths:
    year = int(re.search(r'(\d{4})\.csv$', path).group(1))
    dfs[year] = pd.read_csv(path)
    print(f"Loaded {year}: {dfs[year].shape}")


Mounted at /content/drive
Loaded 2017: (51392, 154)


<ipython-input-155-03358e895310>:16: DtypeWarning: Columns (8,12,13,14,15,16,50,51,52,53,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128) have mixed types. Specify dtype option on import or set low_memory=False.
  dfs[year] = pd.read_csv(path)


Loaded 2018: (98855, 129)
Loaded 2019: (88883, 85)
Loaded 2020: (64461, 61)
Loaded 2021: (83439, 48)
Loaded 2022: (73268, 79)
Loaded 2023: (89184, 84)
Loaded 2024: (65437, 114)


##Cell 2:Harmonizing and Standardizing Columns
This code block aims to clean and prepare the survey data loaded in Cell 1. It deals with the problem of inconsistent column names across different years of surveys. For instance, 'Salary' in one year might be called 'ConvertedComp' in another. This process ensures we have consistent data for analysis.

In [ ]:
# Cell 2 ────────────────────────────────────────────────────────────────────
# Harmonise columns (advanced aliases) & keep only the 39 relevant features

import re, os, numpy as np
from collections import defaultdict

# -------------------------------------------------------------------------
# 1)  Define canonical names and ALL their aliases  (edit freely if needed)
# -------------------------------------------------------------------------
alias_map = {
    # 1  Respondent ID
    "respondent_id" : ["Respondent", "ResponseId"],

    # 2  Country
    "country"       : ["Country"],

    # 3  Employment status
    "employment"    : ["EmploymentStatus", "Employment"],

    # 4  Highest education level
    "education_level" : ["FormalEducation", "EdLevel"],

    # 5  Organisation size
    "org_size"      : ["CompanySize", "OrgSize"],

    # 6  Developer role / type
    "dev_type"      : ["DeveloperType", "DevType"],

    # 7  Total years coding
    "years_code_total" : ["YearsProgram", "YearsCoding", "YearsCode"],

    # 8  Professional years coding
    "years_code_pro"   : ["YearsCodedJob", "YearsCodingProf", "YearsCodePro"],

    # 9  Currency
    "currency"      : ["Currency", "CurrencySymbol"],

    # 10 Compensation (salary / total comp)
    "compensation_total" : ["Salary", "ConvertedSalary",
                            "CompTotal", "ConvertedComp", "ConvertedCompYearly"],

    # 11–18  Tech stacks (worked / desired)
    "langs_worked"       : ["HaveWorkedLanguage", "LanguageWorkedWith",
                            "LanguageHaveWorkedWith"],
    "langs_desired"      : ["WantWorkLanguage", "LanguageDesireNextYear",
                            "LanguageWantToWorkWith"],

    "db_worked"          : ["HaveWorkedDatabase", "DatabaseWorkedWith",
                            "DatabaseHaveWorkedWith"],
    "db_desired"         : ["WantWorkDatabase", "DatabaseDesireNextYear",
                            "DatabaseWantToWorkWith"],

    "platform_worked"    : ["HaveWorkedPlatform", "PlatformWorkedWith",
                            "PlatformHaveWorkedWith"],
    "platform_desired"   : ["WantWorkPlatform", "PlatformDesireNextYear",
                            "PlatformWantToWorkWith"],

    "webframe_worked"    : ["HaveWorkedFramework", "FrameworkWorkedWith",
                            "WebFrameWorkedWith", "WebframeWorkedWith",
                            "WebframeHaveWorkedWith"],
    "webframe_desired"   : ["WantWorkFramework", "FrameworkDesireNextYear",
                            "WebFrameDesireNextYear", "WebframeDesireNextYear",
                            "WebframeWantToWorkWith"],

    # 19 Survey length
    "survey_length"      : ["SurveyLong", "SurveyTooLong", "SurveyLength"],

    # 20 Operating system
    "operating_system"   : ["OperatingSystem", "OpSys",
                            "OpSysPersonal use", "OpSysProfessional use"],

    # 21–24  Stack Overflow usage
    "so_visit_freq"  : ["StackOverflowVisit", "SOVisitFreq"],
    "so_account"     : ["StackOverflowHasAccount", "SOAccount"],
    "so_part_freq"   : ["StackOverflowParticipate", "SOPartFreq"],
    "so_community"   : ["StackOverflowCommunity", "SOComm"],

    # 25–28  Demographics
    "age"            : ["Age"],
    "survey_ease"    : ["SurveyEasy", "SurveyEase"],
    "gender"         : ["Gender"],
    "ethnicity"      : ["Race", "RaceEthnicity", "Ethnicity"],

    # 29  Main branch
    "main_branch"    : ["MainBranch"],

    # 30–31  Misc tech
    "misc_tech_worked"  : ["MiscTechWorkedWith", "MiscTechHaveWorkedWith"],
    "misc_tech_desired" : ["MiscTechDesireNextYear", "MiscTechWantToWorkWith"],

    # 32–33  Satisfaction
    "job_satisfaction"     : ["JobSatisfaction", "JobSat"],
    "career_satisfaction"  : ["CareerSatisfaction", "CareerSat"],

    # 34  Open-source participation
    "open_source"       : ["OpenSource", "OpenSourcer"],

    # 35  Learning resources
    "learning_resources": ["EducationTypes", "SelfTaughtTypes", "EduOther",
                           "NEWLearn", "LearnCode", "LearnCodeOnline",
                           "LearnCodeCoursesCert"],

    # 36  Time since last job change
    "last_job_change"   : ["LastNewJob", "LastHireDate",
                           "NEWJobHunt", "NEWOnboardGood"],

    # 37  Job factors importance
    "job_factors"       : ["JobFactors"] + [f"AssessJob{i}" for i in range(1, 11)],

    # 38  Undergraduate major
    "undergrad_major"   : ["MajorUndergrad", "UndergradMajor"],

    # 39  Work-life balance
    "work_life_balance" : ["ExCoderBalance", "BetterLife"],
}

# Quick lookup: alias → canonical
alias_to_canon = {alias: canon
                  for canon, aliases in alias_map.items()
                  for alias in aliases}

# Also map canonical names to themselves (in case some files already use them)
for canon in alias_map.keys():
    alias_to_canon.setdefault(canon, canon)

# -------------------------------------------------------------------------
# 2)  Consolidate all aliases inside each dataframe
# -------------------------------------------------------------------------
def consolidate_aliases(df):
    df = df.copy()

    # Gather actual columns per canonical
    present_by_canon = defaultdict(list)
    for col in df.columns:
        canon = alias_to_canon.get(col)
        if canon:
            present_by_canon[canon].append(col)

    # Merge columns row-wise (first non-NA wins)
    for canon, cols in present_by_canon.items():
        if len(cols) == 1 and cols[0] == canon:
            # Already canonical and unique → nothing to do
            continue

        merged = df[cols[0]].copy()
        for alt in cols[1:]:
            merged = merged.combine_first(df[alt])

        # If canonical name itself exists separately, merge it too
        if canon in df.columns and canon not in cols:
            merged = df[canon].combine_first(merged)

        df[canon] = merged
        # Drop all non-canonical alias cols we just merged
        drop = [c for c in cols if c != canon]
        df.drop(columns=drop, inplace=True)

    return df

for year in dfs:
    dfs[year] = consolidate_aliases(dfs[year])

# -------------------------------------------------------------------------
# 3)  Keep ONLY the 39 canonical columns we care about
# -------------------------------------------------------------------------
CANON_COLS = list(alias_map.keys())   # preserve order for convenience

for year, df in dfs.items():
    # Ensure every desired column exists (create empty if missing)
    for col in CANON_COLS:
        if col not in df.columns:
            df[col] = np.nan

    dfs[year] = df.loc[:, CANON_COLS].copy()
    print(f"{year}: kept {dfs[year].shape[1]:>2} cols  →  {dfs[year].shape}")



2017: kept 39 cols  →  (51392, 39)
2018: kept 39 cols  →  (98855, 39)
2019: kept 39 cols  →  (88883, 39)
2020: kept 39 cols  →  (64461, 39)
2021: kept 39 cols  →  (83439, 39)
2022: kept 39 cols  →  (73268, 39)
2023: kept 39 cols  →  (89184, 39)
2024: kept 39 cols  →  (65437, 39)


##Saving the Cleaned DataFrames
This part of the code is responsible for saving the cleaned dataframes back to your Google Drive. This is important for persisting the changes made to the data and for using the cleaned data in later analyses.

In [ ]:
# -------------------------------------------------------------------------
# 4)  Save cleaned dataframes back to Drive
# -------------------------------------------------------------------------
CLEAN_DIR = os.path.join(DATA_DIR, "clean_39cols")
os.makedirs(CLEAN_DIR, exist_ok=True)

for year, df in dfs.items():
    out_path = os.path.join(CLEAN_DIR, f"{year}_clean39.csv")
    df.to_csv(out_path, index=False)

print(f"\n✅ All cleaned files written to: {CLEAN_DIR}")


✅ All cleaned files written to: /content/drive/MyDrive/DataScienceLabProject/clean_39cols


##Cell 3: Data Cleaning and Preparation
This code block focuses on further cleaning the survey data and preparing it for analysis by:

Dropping Empty Columns:
It iterates through each year's data (dfs) and identifies columns that are completely empty (contain only NaN - Not a Number - values).
Using dfs[year].isnull().all(), it checks if all values in a column are null.
If a column is found to be empty, it's removed from the DataFrame for that year using dfs[year].drop(columns=empty_cols).

In [ ]:


import pandas as pd
# Cell 3 ────────────────────────────────────────────────────────────────────
# Drop empty columns, low information columns, and convert to numeric

# --- 1. Drop empty columns ---
for year in dfs:
    # Identify columns with all NaN values
    empty_cols = dfs[year].columns[dfs[year].isnull().all()]
    print(f"Year {year}: Dropping {len(empty_cols)} completely empty columns")
    dfs[year] = dfs[year].drop(columns=empty_cols)

# --- 2. Drop low information columns ---
LOW_INFORMATION_THRESHOLD = 0.05  # 5%
for year in dfs:
    # Calculate the percentage of non-missing values per column
    information_percentages = dfs[year].notna().sum() / len(dfs[year])

    # Identify low-information columns
    low_info_cols = information_percentages[
        information_percentages < LOW_INFORMATION_THRESHOLD
    ].index

    print(f"Year {year}: Dropping {len(low_info_cols)} low information columns")
    dfs[year] = dfs[year].drop(columns=low_info_cols)

# --- 3. Convert to numeric ---
for year in dfs:
    # Select columns that are not 'respondent_id' or 'country'
    numeric_cols = dfs[year].select_dtypes(exclude=['object']).columns
    numeric_cols = numeric_cols[numeric_cols != 'respondent_id']
    numeric_cols = numeric_cols[numeric_cols != 'country']

    for col in numeric_cols:
      try:
        dfs[year][col] = pd.to_numeric(dfs[year][col], errors='coerce')
      except Exception as e:
        print(f"Error converting {col} in {year}: {e}")

# --- 4. (Optional) Save cleaned dataframes back to Drive ---
CLEAN_DIR = os.path.join(DATA_DIR, "clean_numeric")
os.makedirs(CLEAN_DIR, exist_ok=True)

for year, df in dfs.items():
    out_path = os.path.join(CLEAN_DIR, f"{year}_clean_numeric.csv")
    df.to_csv(out_path, index=False)

print(f"\n✅ All cleaned (numeric) files written to: {CLEAN_DIR}")


Year 2017: Dropping 11 completely empty columns
Year 2018: Dropping 5 completely empty columns
Year 2019: Dropping 0 completely empty columns
Year 2020: Dropping 3 completely empty columns
Year 2021: Dropping 7 completely empty columns
Year 2022: Dropping 7 completely empty columns
Year 2023: Dropping 9 completely empty columns
Year 2024: Dropping 8 completely empty columns
Year 2017: Dropping 1 low information columns
Year 2018: Dropping 0 low information columns
Year 2019: Dropping 0 low information columns
Year 2020: Dropping 0 low information columns
Year 2021: Dropping 0 low information columns
Year 2022: Dropping 0 low information columns
Year 2023: Dropping 0 low information columns
Year 2024: Dropping 0 low information columns

✅ All cleaned (numeric) files written to: /content/drive/MyDrive/DataScienceLabProject/clean_numeric


##Cell 4: Full Numerisation and One-Hot Encoding
This cell aims to convert the remaining text-based or categorical data into numerical format, which is often necessary for machine learning algorithms. It uses two main techniques:

Numerisation/Ordinal Mapping: Converting categories into ordered numbers (e.g., education levels: 'primary' → 8, 'bachelor' → 16, 'master' → 18, etc.).

One-Hot Encoding: Creating new binary (0/1) columns for each category within multi-select fields (e.g., programming languages worked with: a column for Python, one for JavaScript, etc.).

In [ ]:
# ╔════════════════════════════════════════════════════════════════╗
# ║  Efficient numerisation & one-hot for 2017-2024 SO surveys     ║
# ╚════════════════════════════════════════════════════════════════╝
import numpy as np, pandas as pd, re, os
from collections import Counter

TOP_N, RARE_THRESHOLD = 15, 30
DATA_DIR = globals().get("DATA_DIR", "/content")
ENC_DIR  = f"{DATA_DIR}/clean_numeric_encoded"
os.makedirs(ENC_DIR, exist_ok=True)

# ── helper regex + dictionaries ──────────────────────────────────
ORG_RE = re.compile(r"(\d+)[^\d]+(\d+)")
NUM_RE = re.compile(r"\d+(\.\d+)?")          # for years_code
EDU_MAP = {
    "primary": 8, "elementary": 8, "secondary": 12, "high school": 12,
    "associate": 14, "bachelor": 16, "master": 18,
    "professional": 20, "doctor": 21,
}
SURVEY_LEN_MAP = {"reasonable": 1, "too long": 2, "way too long": 3}
VISIT_MAP = {"never": 0, "less": 1, "monthly": 2, "weekly": 3,
             "daily": 4, "multiple": 5}
ACCOUNT_MAP, PART_MAP, EASE_MAP = {"yes": 1, "no": 0}, VISIT_MAP, {
    "very easy": 5, "easy": 4, "neither": 3,
    "difficult": 2, "very difficult": 1,
}

MULTI_COLS = [
    "dev_type", "langs_worked", "langs_desired",
    "db_worked", "db_desired",
    "platform_worked", "platform_desired",
    "webframe_worked", "webframe_desired",
    "misc_tech_worked", "misc_tech_desired",
    "learning_resources", "job_factors",
    "gender", "ethnicity",
]

# ── 1) Build global token frequency (handles missing columns) ────
concat = pd.concat(
    [df.reindex(columns=MULTI_COLS) for df in dfs.values()],
    axis=0, ignore_index=True
)
token_ctr = {
    c: Counter(
        tok.strip().lower()
        for v in concat[c].dropna().astype(str)
        for tok in v.split(";")
    )
    for c in MULTI_COLS
}
TOP_TOK = {c: [t for t, _ in ctr.most_common(TOP_N)] for c, ctr in token_ctr.items()}

# ── 2) Fast converters ───────────────────────────────────────────
def org_to_mid(s):
    if pd.isna(s): return np.nan
    st = str(s).lower()
    if "fewer" in st: return float(re.search(r"\d+", st)[0]) / 2
    if "more"  in st or "10000" in st: return 2e4
    m = ORG_RE.search(st)
    return (int(m[1]) + int(m[2])) / 2 if m else np.nan

def years_code(s):
    if pd.isna(s): return np.nan
    st = str(s).lower()
    if "less" in st: return 0
    if "more" in st: return 51
    m = NUM_RE.search(st)
    return float(m.group()) if m else np.nan

def map_partial(series, mapper):
    return series.str.lower().map(mapper)

def onehot(df, col):
    """Return dummy columns for TOP_TOK[col], guaranteeing no duplicates."""
    if col not in df:
        return pd.DataFrame(index=df.index)

    # always treat the column as text
    s = df[col].fillna("").astype(str)

    # split multi-select answers, lowercase & strip
    d = s.str.get_dummies(";")
    d.columns = d.columns.str.strip().str.lower()

    # Collapse tokens that became identical after cleaning
    if d.columns.duplicated().any():
        d = d.T.groupby(level=0).max().T      # avoids the FutureWarning

    # Ensure *every* top-N token exists (missing → 0)
    d = d.reindex(columns=TOP_TOK[col], fill_value=0)

    # Prefix for uniqueness across questions
    d.columns = [f"{col}__{c}" for c in d.columns]
    return d



# ── 3) Encode every year ─────────────────────────────────────────
encoded_dfs = {}
for yr, df in dfs.items():
    df = df.copy()

    # numeric maps
    for col in ("years_code_total", "years_code_pro"):
        if col in df: df[col] = df[col].map(years_code)
    if "org_size" in df: df["org_size"] = df["org_size"].map(org_to_mid)
    if "education_level" in df:
        df["education_level"] = (
            df["education_level"].str.lower()
            .apply(lambda x: next((v for k, v in EDU_MAP.items()
                                   if pd.notna(x) and k in x), np.nan))
        )
    if "survey_length" in df:
        df["survey_length"] = map_partial(df["survey_length"], SURVEY_LEN_MAP)
    for col, mp in (("so_visit_freq", VISIT_MAP), ("so_part_freq", PART_MAP),
                    ("so_account", ACCOUNT_MAP), ("survey_ease", EASE_MAP)):
        if col in df: df[col] = map_partial(df[col], mp)

    # age band midpoint
    if "age" in df and df["age"].dtype == "O":
        df["age"] = pd.to_numeric(df["age"].str.extract(r"(\d+)")[0], errors="coerce")

    # multi-select one-hot
    multi_df = pd.concat([onehot(df, c) for c in MULTI_COLS], axis=1)
    multi_df = multi_df.loc[:, multi_df.sum() >= RARE_THRESHOLD]

    # drop raw text cols & add dummies
    df = df.drop(columns=[c for c in MULTI_COLS if c in df]).join(multi_df)

    # encode residual object columns (except respondent_id / country)
    EXEMPT_OBJ = {"respondent_id", "country", "currency"}  # ← added currency
    for c in df.select_dtypes("object"):
      if c in EXEMPT_OBJ:
          continue
      df[c] = df[c].astype("category").cat.codes.replace(-1, np.nan)


    encoded_dfs[yr] = df
    df.to_csv(f"{ENC_DIR}/{yr}_encoded.csv", index=False)
    print(f"{yr}: {df.shape}")

print("✅ Encoded CSVs in", ENC_DIR)


2017: (51392, 83)
2018: (98855, 124)
2019: (88883, 172)
2020: (64461, 168)
2021: (83439, 151)
2022: (73268, 162)
2023: (89184, 145)
2024: (65437, 146)
✅ Encoded CSVs in /content/drive/MyDrive/DataScienceLabProject/clean_numeric_encoded


In [ ]:
# ╔══════════════════════════════════════════════╗
# ║  Cell 4-bis : FX  &  CPI lookup dictionaries  ║
# ╚══════════════════════════════════════════════╝
import pandas as pd

# --- 1.  Average yearly FX to USD (mid-market) ---
# Numbers below are illustrative.  Replace / extend as you like.
FX_DICT = {
    2017: {'USD':1, 'EUR':1.13, 'GBP':1.29, 'INR':0.015},
    2018: {'USD':1, 'EUR':1.18, 'GBP':1.34, 'INR':0.014},
    2019: {'USD':1, 'EUR':1.12, 'GBP':1.28, 'INR':0.014},
    2020: {'USD':1, 'EUR':1.14, 'GBP':1.28, 'INR':0.013},
    2021: {'USD':1, 'EUR':1.18, 'GBP':1.38, 'INR':0.013},
    2022: {'USD':1, 'EUR':1.05, 'GBP':1.24, 'INR':0.012},
    2023: {'USD':1, 'EUR':1.08, 'GBP':1.24, 'INR':0.012},
    2024: {'USD':1, 'EUR':1.09, 'GBP':1.27, 'INR':0.012},
}

fx_rates = (
    pd
    .DataFrame([
        {'year': y, 'currency_iso': cur, 'usd_rate': rate}
        for y, mp in FX_DICT.items()
        for cur, rate in mp.items()
    ])
)

# --- 2.  CPI index (2024 = 100) ---
CPI = {2017:86.0, 2018:88.1, 2019:89.7, 2020:91.6,
       2021:94.0, 2022:97.1, 2023:98.9, 2024:100.0}

cpi_factor = pd.Series(
    {y: CPI[2024]/v for y, v in CPI.items()},
    name='cpi_factor'
)
print("✅  FX & CPI lookups built in-memory")


✅  FX & CPI lookups built in-memory


In [ ]:
# ╔══════════════════════════════════════════════════════════════╗
# ║  Cell 5 : add salary_usd_nom  &  salary_usd_2024 to each df  ║
# ╚══════════════════════════════════════════════════════════════╝
import re, numpy as np

ISO_RE   = re.compile(r"\b[A-Z]{3}\b")
SPECIALS = {
    "u.s. dollars": "USD", "us dollars": "USD", "u.s. dollar": "USD",
    "euros": "EUR", "british pounds sterling": "GBP",
    "south african rands": "ZAR", "canadian dollars": "CAD",
    "australian dollars": "AUD", "polish złoty": "PLN",
    "polish zloty": "PLN", "indian rupees": "INR",
    "russian rubles": "RUB", "bitcoin": "BTC",
    "non": np.nan, "none": np.nan,
}

def to_iso(cur):
    if pd.isna(cur):
        return np.nan
    s = str(cur).lower().strip()
    for k, v in SPECIALS.items():
        if k in s:
            return v
    m = ISO_RE.search(s.upper())   # ← use s here
    return m.group() if m else np.nan


for yr, df in encoded_dfs.items():
    df = df.copy()

    # a) currency → ISO
    df["currency_iso"] = df["currency"].apply(to_iso)

    # b) left-join FX for that year
    fx_year = fx_rates.query("year == @yr")
    df = df.merge(fx_year, on="currency_iso", how="left")

    # c) salary conversions
    df["salary_usd_nom"] = np.where(
        df["currency_iso"] == "USD",
        df["compensation_total"],                # already USD
        df["compensation_total"] * df["usd_rate"]   # convert
    )

    df["salary_usd_2024"] = df["salary_usd_nom"] * cpi_factor.loc[yr]

    # d) push back into dict (no writing to disk)
    encoded_dfs[yr] = df
    print(f"{yr}:  added salary columns")

print("🎯  All years converted (nominal & real 2024 USD)")


2017:  added salary columns
2018:  added salary columns
2019:  added salary columns
2020:  added salary columns
2021:  added salary columns
2022:  added salary columns
2023:  added salary columns
2024:  added salary columns
🎯  All years converted (nominal & real 2024 USD)


Drop empty columns from each year.

In [ ]:
# prompt: Check if any encoded_dfs[year] has completely empty columns, if so, drop them

for year in encoded_dfs:
    # Identify columns with all NaN values
    empty_cols = encoded_dfs[year].columns[encoded_dfs[year].isnull().all()]
    if len(empty_cols) > 0:
        print(f"Year {year}: Dropping {len(empty_cols)} completely empty columns from encoded_dfs")
        encoded_dfs[year] = encoded_dfs[year].drop(columns=empty_cols)


Year 2017: Dropping 1 completely empty columns from encoded_dfs
Year 2018: Dropping 3 completely empty columns from encoded_dfs
Year 2019: Dropping 2 completely empty columns from encoded_dfs
Year 2020: Dropping 2 completely empty columns from encoded_dfs
Year 2021: Dropping 2 completely empty columns from encoded_dfs
Year 2022: Dropping 2 completely empty columns from encoded_dfs
Year 2023: Dropping 2 completely empty columns from encoded_dfs
Year 2024: Dropping 2 completely empty columns from encoded_dfs


Transform Country to median salary by year

In [ ]:
#  run this in a new cell (only once per runtime)
!pip install -q pycountry


In [ ]:
# ╔══════════════════════════════════════════════╗
# ║  Cell 6 : country-level median salary panel   ║
# ╚══════════════════════════════════════════════╝
import pandas as pd, requests, io, numpy as np, pycountry

URL = "https://ourworldindata.org/grapher/daily-median-income.csv"
med = pd.read_csv(io.BytesIO(requests.get(URL).content))

# ── 1. keep survey years only ─────────────────────────────────────────
med = med.query("Year >= 2017 & Year <= 2024")

# ── 2. identify the income-value column dynamically ───────────────────
value_col = next(c for c in med.columns if c not in ("Entity", "Code", "Year"))
med = med.rename(columns={
    "Entity": "country",
    "Code":   "iso3",
    "Year":   "year",
    value_col: "med_ppp_2017"          # daily PPP-$ (base-year 2017)
})

# ── 3. daily → annual, PPP-2017 → nominal-2024 USD  ───────────────────
med["med_ppp_2017_year"] = med["med_ppp_2017"] * 365        # days → year
cpi_ratio = CPI[2024] / CPI[2017]                           # from your Cell 4-bis
med["med_usd_2024"] = med["med_ppp_2017_year"] * cpi_ratio  # rough PPP→nominal

# quick sanity check
print(med.head(3))


    country iso3  year  med_ppp_2017 990177-annotations  med_ppp_2017_year  \
8   Albania  ALB  2017     10.187669                NaN        3718.499185   
9   Albania  ALB  2018     11.639976                NaN        4248.591422   
10  Albania  ALB  2019     11.611524                NaN        4238.206260   

    med_usd_2024  
8    4323.836262  
9    4940.222584  
10   4928.146814  


In [ ]:
# ╔══════════════════════════════════════════════════╗
# ║  Cell 7 : attach country median & compute ratios  ║
# ╚══════════════════════════════════════════════════╝
import pandas as pd, numpy as np
from functools import lru_cache
import pycountry        # ← already installed or pip-installed above

@lru_cache
def name_to_iso3(name):
    """convert free-text country → ISO-3; NaN → NaN"""
    if pd.isna(name):
        return np.nan            # <-- skip missing entries
    try:
        return pycountry.countries.search_fuzzy(str(name))[0].alpha_3
    except LookupError:
        return np.nan            # <-- unknown spelling

for yr, df in encoded_dfs.items():
    df = df.copy()

    # a) add ISO-3 code
    df["iso3"] = df["country"].map(name_to_iso3)

    # b) join the median-income table for this year
    med_yr = med.query("year == @yr")[["iso3", "med_usd_2024"]]
    df = df.merge(med_yr, on="iso3", how="left")

    # c) new columns
    df.rename(columns={"med_usd_2024": "country_median_usd_2024"}, inplace=True)
    df["salary_vs_country_median"] = (
        df["salary_usd_2024"] / df["country_median_usd_2024"]
    )

    encoded_dfs[yr] = df
    print(f"{yr}: median added & ratio computed")

print("Done – use ‘salary_vs_country_median’ for downstream models")


2017: median added & ratio computed
2018: median added & ratio computed
2019: median added & ratio computed
2020: median added & ratio computed
2021: median added & ratio computed
2022: median added & ratio computed
2023: median added & ratio computed
2024: median added & ratio computed
Done – use ‘salary_vs_country_median’ for downstream models


In [ ]:
encoded_dfs[2021][
    ["country","salary_usd_2024","country_median_usd_2024","salary_vs_country_median"]
].head()


,country,salary_usd_2024,country_median_usd_2024,salary_vs_country_median
0,Slovakia,6025.531915,9964.628174,0.604692
1,Netherlands,NaN,25348.954180,NaN
2,Russian Federation,NaN,NaN,NaN
3,Austria,NaN,24725.999767,NaN
4,United Kingdom of Great Britain and Northern I...,NaN,19892.547959,NaN


In [ ]:
# prompt: drop the country column from all years and display the head of some years

for year in encoded_dfs:
    if 'country' in encoded_dfs[year].columns:
        encoded_dfs[year] = encoded_dfs[year].drop(columns=['country'])
for year in encoded_dfs:
    if 'currency' in encoded_dfs[year].columns:
        encoded_dfs[year] = encoded_dfs[year].drop(columns=['currency'])

# Verify that 'currency' column is dropped
for year in encoded_dfs:
    print(f"Year {year}: 'currency' column exists: {'currency' in encoded_dfs[year].columns}")

for year in encoded_dfs:
    if 'iso3' in encoded_dfs[year].columns:
        encoded_dfs[year] = encoded_dfs[year].drop(columns=['iso3'])

# Verify that 'iso3' column is dropped
for year in encoded_dfs:
    print(f"Year {year}: 'iso3' column exists: {'iso3' in encoded_dfs[year].columns}")

for year in encoded_dfs:
    if 'currency_iso' in encoded_dfs[year].columns:
        encoded_dfs[year] = encoded_dfs[year].drop(columns=['currency_iso'])

# Verify that 'currency_iso' column is dropped
for year in encoded_dfs:
    print(f"Year {year}: 'currency_iso' column exists: {'currency_iso' in encoded_dfs[year].columns}")




Year 2017: 'currency' column exists: False
Year 2018: 'currency' column exists: False
Year 2019: 'currency' column exists: False
Year 2020: 'currency' column exists: False
Year 2021: 'currency' column exists: False
Year 2022: 'currency' column exists: False
Year 2023: 'currency' column exists: False
Year 2024: 'currency' column exists: False
Year 2017: 'iso3' column exists: False
Year 2018: 'iso3' column exists: False
Year 2019: 'iso3' column exists: False
Year 2020: 'iso3' column exists: False
Year 2021: 'iso3' column exists: False
Year 2022: 'iso3' column exists: False
Year 2023: 'iso3' column exists: False
Year 2024: 'iso3' column exists: False
Year 2017: 'currency_iso' column exists: False
Year 2018: 'currency_iso' column exists: False
Year 2019: 'currency_iso' column exists: False
Year 2020: 'currency_iso' column exists: False
Year 2021: 'currency_iso' column exists: False
Year 2022: 'currency_iso' column exists: False
Year 2023: 'currency_iso' column exists: False
Year 2024: 'cur

# EDA
ABove we've transform the data to keep only common features present in 5 years or more ( out of 8 ) and transformed them to numeric

Now we'll start doing some analysis of the data

In [ ]:
  # ╔══════════════════════════════════════════════════════════════╗
# ║  Cell 1 : build df_all  +  quick variable overview helper    ║
# ╚══════════════════════════════════════════════════════════════╝
import pandas as pd, numpy as np

# 1) stack yearly frames → one master DF
df_all = (
    pd.concat(
        {yr: df for yr, df in encoded_dfs.items()},      # keep year key
        names=["survey_year"]
    )
    .reset_index(level=0)                                # bring year into column
)

# 2) helper → tidy variable summary
def var_table(df: pd.DataFrame, top_n: int | None = None) -> pd.DataFrame:
    """Return a table: variable, dtype, % missing, n_unique (optionally top_n)."""
    tbl = (
        pd.DataFrame({
            "dtype":   df.dtypes.astype(str),
            "missing_%": df.isna().mean() * 100,
            "n_unique":  df.nunique(dropna=True)
        })
        .sort_values("missing_%", ascending=False)
    )
    return tbl.head(top_n) if top_n else tbl

# 3) quick sanity prints
print(f"✅ df_all shape: {df_all.shape[0]:,} rows × {df_all.shape[1]} columns")
print(var_table(df_all, top_n=5))


In [ ]:
# ╔══════════════════════════════════════════════════════════════╗
# ║  Cell 2-bis : HARD-CODE columns to remove & drop them        ║
# ╚══════════════════════════════════════════════════════════════╝

# === 1) explicit list of survey-meta / open-text columns =======
COLS_TO_REMOVE = [
    # ---- gender & free-text minorities ----
    "gender__other",
    "gender__gender non-conforming",
    "gender__nonbinary, genderfluid, or gender non-conforming",

    # ---- open feedback / comments ----
    "survey_feedback",
    "additional_comments",
    "extra_feedback",

    # ---- long free-text questions ----
    "job_factors__languages, frameworks, and other tools",
    "job_factors__other",
    "dev_environment__other",

    # ---- sparse work–life perception questions ----
    "work_life_balance",
    "work_life_balance__comment",

    # ---- ethnicity very-sparse dummies ----
    "ethnicity__indian",
    "ethnicity__pakistani",
    "ethnicity__other",

    # ---- any other free-text columns you flagged ----
    # (add more names here if needed)
]

# === 2) drop from master DataFrame ==================================
df_all.drop(columns=COLS_TO_REMOVE, inplace=True, errors="ignore")

# === 3) drop from every yearly DataFrame ============================
for yr, df in encoded_dfs.items():
    encoded_dfs[yr] = df.drop(columns=COLS_TO_REMOVE, errors="ignore")

# === 4) sanity report ===============================================
print(f"🚮  Dropped {len(COLS_TO_REMOVE)} columns.")
print("✅  df_all new shape:", df_all.shape)


In [ ]:
# ╔══════════════════════════════════════════════════════════════╗
# ║  Cell 2 : create & save full variable-overview               ║
# ╚══════════════════════════════════════════════════════════════╝
# 1) build the full table
var_overview = var_table(df_all)          # uses helper from Cell 1

# 2) save for later inclusion in the PDF / appendix
var_overview.to_csv("variable_overview.csv")

# 3) show a glimpse
print("Top-10 variables by missing-%")
display(var_overview.head(10))
print("\n📝  Full table saved → variable_overview.csv")


In [ ]:
from google.colab import files; files.download('variable_overview.csv')

In [ ]:
# ╔══════════════════════════════════════════════════════════════╗
# ║  Cell 7 : filter salary outliers and create df_all_clean     ║
# ╚══════════════════════════════════════════════════════════════╝
import numpy as np, pandas as pd

# 1) look at extreme quantiles (for info only)
q1, q99 = df_all["salary_usd_2024"].quantile([0.01, 0.99])
print(f"1 % quantile ≈ {q1:,.0f}  |  99 % quantile ≈ {q99:,.0f}")

# 2) define a pragmatic band (adjust if needed)
LOWER, UPPER = 1_000, 800_000    # USD (real 2024)

# 3) build a clean copy
mask_valid = df_all["salary_usd_2024"].between(LOWER, UPPER, inclusive="both")
df_all_clean = df_all.loc[mask_valid].copy()

# also regenerate encoded_dfs_clean if you need per-year
encoded_dfs_clean = {
    yr: df.loc[df["salary_usd_2024"].between(LOWER, UPPER, inclusive="both")].copy()
    for yr, df in encoded_dfs.items()
}

# 4) report effect
removed = (~mask_valid).sum()
print(f"🚮  Filtered out {removed:,} rows outside [{LOWER:,}, {UPPER:,}] USD.")
print("✅  df_all_clean shape:", df_all_clean.shape)

print("\nClean salary stats:")
display(df_all_clean["salary_usd_2024"].describe())


Overall salary distribution

In [ ]:
# ╔══════════════════════════════════════════════════════════════╗
# ║  Cell 5 : overall salary_usd_2024 histogram                  ║
# ╚══════════════════════════════════════════════════════════════╝
import matplotlib.pyplot as plt

plt.figure(figsize=(6, 4))
df_all["salary_usd_2024"].dropna().plot.hist(bins=50)
plt.xlabel("Salary (real 2024 USD)")
plt.ylabel("Frequency")
plt.title("Overall Distribution of Real 2024-USD Salaries")
plt.tight_layout()
plt.show()


Year-by-year salary density

In [ ]:
# ╔══════════════════════════════════════════════════════════════╗
# ║  Cell 6 : salary_usd_2024 KDE – one figure per year          ║
# ╚══════════════════════════════════════════════════════════════╝
import matplotlib.pyplot as plt

for yr in sorted(df_all["survey_year"].unique()):
    plt.figure(figsize=(6, 4))
    df_all.query("survey_year == @yr")["salary_usd_2024"].dropna().plot.kde()
    plt.xlabel("Salary (real 2024 USD)")
    plt.ylabel("Density")
    plt.title(f"Salary Density – Survey {yr}")
    plt.tight_layout()
    plt.show()
